<a href="https://colab.research.google.com/github/polina-minaeva/text-classification-nlp/blob/main/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D0%B5%D0%B9_%D0%B8_%D0%B2%D1%8B%D1%8F%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%84%D0%B5%D0%B9%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В доступе имеются данные: 1) твиты с новостями и 2) метка о том, являются новости фейковыми или нет. Попробуем обучить модели машинного обучения определять фейковые новости, чтобы в дальнейшем они могли выявлять недостоверную информацию в интернете

#1. Загрузка библиотек

In [ ]:
#работа с данными
import pandas as pd
import numpy as np
from tqdm import tqdm

#работа с текстом
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from gensim.models.word2vec import Word2Vec

#функционал для моделей
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.optim as optim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#2. Загрузка данных

In [ ]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv #данные

df = pd.read_csv('Constraint_Train.csv') #датафрейм
df.head()

--2024-08-19 11:37:32--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.1s    

2024-08-19 11:37:33 (12.3 MB/s) - ‘Constraint_Train.csv’ saved [1253562/1253562]



,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
df['label'].value_counts(normalize=True) #баланс классов

,proportion
label,
real,0.523364
fake,0.476636


#3. Предобработка данных

In [ ]:
'''создадим векторы слов: для этого сначала токенизируем их, а потом векторизируем'''

sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)] #токенизация твитов

100%|██████████| 6420/6420 [00:01<00:00, 3685.11it/s]


In [ ]:
%time model_tweets = Word2Vec(sentences, workers=4, vector_size=300, min_count=3, window=5, epochs=15) #векторизация слов

CPU times: user 8.02 s, sys: 79 ms, total: 8.1 s
Wall time: 7.19 s


In [ ]:
model_tweets.init_sims() #нормирование векторов

<ipython-input-5-ba5d28606ee3>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_tweets.init_sims() #нормирование векторов


In [ ]:
'''векторизованные слова теперь превратим в эмбеддинги (по 300 чисел) для каждого слова'''

#функция
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.average(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [ ]:
#применение функции
features = [get_text_embedding(text) for text in tqdm(df.tweet)]




  0%|          | 0/6420 [00:00<?, ?it/s]


  4%|▍         | 249/6420 [00:00<00:02, 2481.16it/s]


  8%|▊         | 498/6420 [00:00<00:02, 2297.61it/s]


 11%|█▏        | 731/6420 [00:00<00:02, 2308.29it/s]


 15%|█▌        | 978/6420 [00:00<00:02, 2368.95it/s]


 19%|█▉        | 1216/6420 [00:00<00:02, 2361.15it/s]


 23%|██▎       | 1453/6420 [00:00<00:02, 2248.42it/s]


 26%|██▌       | 1679/6420 [00:00<00:02, 2211.71it/s]


 30%|██▉       | 1901/6420 [00:00<00:02, 2198.66it/s]


 33%|███▎      | 2141/6420 [00:00<00:01, 2259.17it/s]


 37%|███▋      | 2385/6420 [00:01<00:01, 2313.21it/s]


 41%|████      | 2617/6420 [00:01<00:01, 2141.47it/s]


 44%|████▍     | 2834/6420 [00:01<00:01, 2120.42it/s]


 48%|████▊     | 3077/6420 [00:01<00:01, 2206.41it/s]


 52%|█████▏    | 3316/6420 [00:01<00:01, 2254.11it/s]


 55%|█████▌    | 3563/6420 [00:01<00:01, 2313.78it/s]


 59%|█████▉    | 3796/6420 [00:01<00:01, 2149.73it/s]


 63%|██████▎   | 4041/6420 [00:01<00:01, 2233.80it/s]


 66%|█

#4. Разделение данных на тренировочную и тестовую выборки

In [ ]:
labels = (df.label == 'real').astype(int).to_list() #получение категориальных данных в таргете

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

#5. Классификация новостей методом опорных векторов

In [ ]:
supv = SVC(random_state=42)
supv.fit(X_train, y_train)

SVC(random_state=42)

In [ ]:
ypred_supv = supv.predict(X_test)
f1_score(y_test, ypred_supv)

0.9250785105428443

#6. Классификация новостей с помощью бустинга

In [ ]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
ypred_xgb = xgb.predict(X_test)

f1_score(y_test, ypred_xgb, average='macro')

0.924466946764959

#7. Классификация новостей с помощью НС

In [ ]:
'''создадим обычную нейросеть с одним слоем'''

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.out = nn.Linear(300, 1) #принимает эмбеддинг размером 300, а на выходе выдает один символ

    def forward(self, x):
        return torch.sigmoid(self.out(x)) #сигмоида для классификации 0 и 1

net = Net()
net.cuda()
print(net)

Net(
  (out): Linear(in_features=300, out_features=1, bias=True)
)


In [ ]:
'''создаем тензоры для пайторч'''

in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

<ipython-input-21-2021dd2f196b>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  in_data = torch.tensor(X_train).float()


In [ ]:
'''оптимизируем модель'''

optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

  0%|          | 0/6420 [03:58<?, ?it/s]


In [ ]:
'''пишем функцию для эпох'''

def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size].cuda()
        batch_y = targets[i:i + batch_size].cuda()
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
'''запускаем обучение'''

for i in range(20):
  train_one_epoch(in_data, targets)

100%|██████████| 269/269 [00:00<00:00, 336.39it/s]


tensor(0.6099, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 269/269 [00:00<00:00, 995.91it/s]


tensor(0.5963, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 269/269 [00:00<00:00, 996.77it/s] 


...

tensor(0.5287, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 269/269 [00:00<00:00, 1067.75it/s]


tensor(0.5272, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 269/269 [00:00<00:00, 1083.02it/s]

tensor(0.5257, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [ ]:
'''проверяем модель на тестовых данных'''

in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net(in_data_test.cuda()).squeeze(1)

In [ ]:
result = (output.cpu() > 0.5) == targets_test
result.sum().item() / len(result)

0.8456819254365266

Как итог, качество моделей бустинга и SVC составило более 0.91. А качество модели нейросети составило более 0.84. В целом, моедли хорошо справились с задачей предсказания фейков